In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import json
import re

PATH = "Datasets/"

In [14]:
#food_emissions = pd.read_csv(PATH+"Food_Product_Emissions.csv")
#food_emissions.drop(['Unit of GHG Emissions'],axis=1,inplace=True)

In [40]:
meat_prod = pd.read_csv(PATH+"meat_prod_codename.csv",sep=",")
meat_prod = meat_prod.drop_duplicates(subset=['Entity'])

In [41]:
meat_prod.columns
meat_prod.to_json('codename.json',orient="columns")

In [5]:
meat_prod.fillna(0,inplace=True)

In [7]:
def getLastYear(df,entities,nb_year=0):
   new_df = pd.DataFrame()
   for e in entities:
      #remove bad country
      if len(e.split("(")) < 2:
         _lastYear = df[df['Entity'] == e].iloc[-1]
         if _lastYear['Year'] == max(np.unique(meat_prod.Year)):
            if nb_year:
               _lastYear = df[df['Entity'] == e].iloc[-nb_year:]
               if(min(_lastYear.Year) == min(meat_prod[meat_prod['Entity'] == "France"].iloc[-10:].Year) and len(_lastYear) == nb_year):
                  new_df = new_df.append(_lastYear)
            else:
               new_df = new_df.append(_lastYear)
   return new_df
   


In [8]:
meat_prod_last_year = getLastYear(meat_prod,np.unique(meat_prod['Entity']))

/tmp/ipykernel_23044/4170374347.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append(_lastYear)
/tmp/ipykernel_23044/4170374347.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append(_lastYear)
/tmp/ipykernel_23044/4170374347.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append(_lastYear)
/tmp/ipykernel_23044/4170374347.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append(_lastYear)
/tmp/ipykernel_23044/4170374347.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat inste

## Production Datasets

In [9]:
def convertProd(df):
    #irrelevent meta data info
    df.drop(['Code'],axis=1,inplace=True)

    #dropping every columns that refer to the tonnes
    df.drop(['Meat, Total | tonnes', 'Game | tonnes',
        'Duck | tonnes', 'Horse | tonnes', 'Camel | tonnes',
        'Goose and guinea fowl | tonnes', 'Sheep and goat | tonnes',
        'Beef and buffalo | tonnes', 'Pig | tonnes', 'Poultry | tonnes'],axis=1,inplace=True)

    #dropping turkey because almost useless
    df.drop(['Turkey | Produced'],axis=1,inplace=True)
    #adding total animals produced
    df['Meat, Total | Produced'] = df.sum(axis=1,numeric_only=True)
    return df


In [10]:
meat_prod_last_year = convertProd(getLastYear(meat_prod,np.unique(meat_prod['Entity'])).copy()).drop(['Year'],axis=1)

In [11]:
meat_prod_10_last_years = convertProd(getLastYear(meat_prod,np.unique(meat_prod['Entity']),10).copy())

In [12]:
meat_prod_last_year.to_csv("meat_prod_last_year.csv",sep=";")
meat_prod_10_last_years.to_csv("meat_prod_10_years.csv",sep=";")

In [13]:
meat_prod_last_year.set_index('Entity').to_json("meat_prod_last_year.json",orient='index')

In [14]:
entities = np.unique(meat_prod_10_last_years.Entity)
res = {}
for e in entities:
    res[e] = {}
    for _, row in meat_prod_10_last_years[meat_prod_10_last_years['Entity'] == e].iterrows():
        res[e][row.Year] = {
                                "Meat, Total | Produced":       row["Meat, Total | Produced"],
                                "Cattle | Produced":            row["Cattle | Produced"],
                                "Goat | Produced":              row["Goat | Produced"],
                                "Chicken | Produced":           row["Chicken | Produced" ],
                                "Pig | Produced":               row["Pig | Produced"],
                                "Lamb and mutton | Produced":   row["Lamb and mutton | Produced"]
                            }

In [15]:
with open("meat_prod_10_last_years.json", "w") as fp:
    json.dump(res,fp) 

## Tonnes Datasets

In [16]:
def convertTonnes(df):
   #irrelevent meta data info
   df.drop(['Code'],axis=1,inplace=True)

   #dropping every columns that refer to the Production
   df.drop(['Cattle | Produced', 'Goat | Produced',
      'Chicken | Produced', 'Turkey | Produced', 'Pig | Produced',
      'Lamb and mutton | Produced',],axis=1,inplace=True)

   #dropping because almost useless or too specific
   df.drop(['Game | tonnes',
      'Horse | tonnes', 'Camel | tonnes',],axis=1,inplace=True)
   
   #computing chicken from poultry 
   df['Chicken | tonnes'] = df['Poultry | tonnes'] - (df['Duck | tonnes'] + df['Goose and guinea fowl | tonnes'])
   df.drop(['Poultry | tonnes'],axis=1,inplace=True)

   #recomputing total animals in tonnes because we removed some
   
   df['Meat, Total | tonnes'] = df.drop(['Meat, Total | tonnes'],axis=1).sum(axis=1,numeric_only=True)
   
   return df

In [17]:
meat_tonnes_last_year = convertTonnes(getLastYear(meat_prod,np.unique(meat_prod['Entity'])).copy()).drop(['Year'],axis=1)


In [18]:
meat_tonnes_10_last_years = convertTonnes(getLastYear(meat_prod,np.unique(meat_prod['Entity']),10).copy())


In [19]:
meat_tonnes_last_year.to_csv("meat_tonnes_last_year.csv",sep=";")
meat_tonnes_10_last_years.to_csv("meat_tonnes_10_years.csv",sep=";")

In [20]:
meat_tonnes_last_year.set_index('Entity').to_json("meat_tonnes_last_year.json",orient='index')

In [21]:
entities = np.unique(meat_tonnes_10_last_years.Entity)
res = {}
for e in entities:
    res[e] = {}
    for _, row in meat_tonnes_10_last_years[meat_tonnes_10_last_years['Entity'] == e].iterrows():
        res[e][row.Year] = {
                                "Meat, Total | tonnes":             row["Meat, Total | tonnes"],
                                "Duck | tonnes":                    row["Duck | tonnes"],
                                "Goose and guinea fowl | tonnes":   row["Goose and guinea fowl | tonnes"],
                                "Sheep and goat | tonnes":          row["Sheep and goat | tonnes"],
                                "Beef and buffalo | tonnes":        row["Beef and buffalo | tonnes"],
                                "Pig | tonnes":                     row["Pig | tonnes"],
                                "Chicken | tonnes ":                row["Chicken | tonnes"]
                            }

In [22]:
with open("meat_tonnes_10_last_years.json", "w") as fp:
    json.dump(res,fp) 